<a href="https://colab.research.google.com/github/m109103/bert/blob/main/MyBert6360_k10_200.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-bert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for keras-bert: filename=keras_bert-0.89.0-py3-none-any.whl size=33517 sha256=5c7f8f519c2ca81f745692c27868841f396d26450150bbdea1dd9635bb0eb4cf
  Stored in directory: /root/.cache/pip/wheels/a4/e8/45/842b3a39831261aef9154b907eacbc4ac99499a99ae829b06f
  Created wheel for keras-transformer: filename=keras_transformer-0.40.0-py3-none-any.whl size=12305 sha256=ea06d6c1d4860113524a159b0e39f1e2dc00334064c6b21875520396c0c8b60d
  Stored in directory: /root/.cache/pip/wheels/46/68/26/692ed21edd832833c3b0a0e21615bcacd99ca458b3f9ed571f
  Created wheel for keras-embed-sim: filename=keras_embed_sim-0.10.0-py3-none-any.whl size=3960 sha256=3cb86cc9dca6627ddc303a47cdc202be47b4a6602f40e857f7b507d1d132f47b
  Stored in directory: /root/.cache/pip/wheels/81/67/b5/d847588d075895281e1cf5590f819bd4cf076a554872268bd5
  Created wheel for keras-layer-normalization: filename=keras_layer_normalizatio

In [2]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
1.4615392880000115
GPU (s):
0.037175576999999294
GPU speedup over CPU: 39x


In [3]:
from google.colab import drive  
drive.mount('/content/gdrive')  
%cd /content/gdrive/"My Drive"/"Colab Notebooks"/"bert6360_200"

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/bert6360_200


In [4]:
import sys
import random
import numpy as np

from tensorflow.keras.layers import *
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam

from keras_bert import (
    load_vocabulary,
    load_trained_model_from_checkpoint,
    Tokenizer,
    get_checkpoint_paths,
)
from keras_bert.datasets import get_pretrained, PretrainedList

In [ ]:
# 指定訓練資料與測試資料的比例
BATCH_SIZE = 10 # batch size建議不要設得太大，不然很有可能out of memory
EPOCHS = 5 # epoch 5次其實就很夠了，當然你可以嘗試再大一點，只是訓練要更久

In [5]:
model_path = get_pretrained(PretrainedList.chinese_base)
paths = get_checkpoint_paths(model_path)
bert_model = load_trained_model_from_checkpoint(
    paths.config, paths.checkpoint, training=False, seq_len=None
)
bert_model.summary(line_length=120)

381902848/381892918 [==============================] - 3s 0us/step
Model: "model_1"
________________________________________________________________________________________________________________________
 Layer (type)                          Output Shape               Param #       Connected to                            
 Input-Token (InputLayer)              [(None, None)]             0             []                                      
                                                                                                                        
 Input-Segment (InputLayer)            [(None, None)]             0             []                                      
                                                                                                                        
 Embedding-Token (TokenEmbedding)      [(None, None, 768),        16226304      ['Input-Token[0][0]']                   
                                        (21128, 768)]                

In [6]:
token_dict = load_vocabulary(paths.vocab)
len(token_dict)
tokenizer = Tokenizer(token_dict)

載入打散後資料

In [ ]:
import pickle
with open('x_train.pickle', 'rb') as f:
    x_text = pickle.load(f)
with open('y_train.pickle', 'rb') as f:
    y_tag = pickle.load(f)

X = x_text[:]
Y = y_tag[:]


In [7]:
#設置模型
def build_model():
    #bert_model.summary(line_length=120)
    model_path = get_pretrained(PretrainedList.chinese_base)
    paths = get_checkpoint_paths(model_path)
    bert_model = load_trained_model_from_checkpoint(
     paths.config, paths.checkpoint, training=False, seq_len=None
    )
    #bert Tokenizer
    token_dict = load_vocabulary(paths.vocab)
    len(token_dict)
    tokenizer = Tokenizer(token_dict)
    # 把 BERT 模型裡的每一層的trainable屬性都設成True，讓BERT模型裡的權重都可以被訓練
    for l in bert_model.layers:
        l.trainable = True

    # 模型的輸入，我們會需要兩個輸入，分別是語句內容以及其對應的segments
    x1_in = Input(shape=(None,))
    x2_in = Input(shape=(None,))
    x = bert_model([x1_in, x2_in])
    x = Lambda(lambda x: x[:, 0])(x) # 取出[CLS]對應位置的向量出來
    p = Dense(18, activation='softmax')(x) # 輸出情緒傾向數值

    model = Model([x1_in, x2_in], p)

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=Adam(1e-5),    #用較小的學習率
                  metrics=['accuracy'])

    model.summary()
    return model

In [ ]:
k=10
num_val_samples = len(X) // k
cvscores = []
results = []
for i in range(k):
    print('processing fold #',i)
    test_data = X[i*num_val_samples:(i+1)*num_val_samples]
    tag_test = Y[i*num_val_samples:(i+1)*num_val_samples]
    train_data = np.concatenate(
        [X[:i*num_val_samples],X[(i+1)*num_val_samples:]],axis=0)
    tag_train = np.concatenate(
        [Y[:i*num_val_samples],Y[(i+1)*num_val_samples:]],axis=0)

    partial_train_data = []
    partial_train_targets = []
    val_data = []
    val_targets = []

    indices = []
    for sentence in train_data:
        ids, segments = tokenizer.encode(sentence, max_len=512) # Tokenizer進行分詞
        indices.append(ids)
    indices = np.array(indices)
    partial_train_data = [indices, np.zeros_like(indices)]
    partial_train_targets = np.array(tag_train)

    indices = []
    for sentence in test_data:
        ids, segments = tokenizer.encode(sentence, max_len=512) # Tokenizer進行分詞
        indices.append(ids)
    indices = np.array(indices)
    val_data = [indices, np.zeros_like(indices)]
    val_targets = np.array(tag_test)
    
    model = build_model()   
    
    history = model.fit(
        partial_train_data,partial_train_targets,
        batch_size=BATCH_SIZE,epochs=EPOCHS,verbose=0)
# evaluate the model
    scores = model.evaluate(val_data,val_targets,batch_size=BATCH_SIZE, verbose=1) 
    result = model.predict(val_data)
    results.append(result)
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
print("End of Predict DataSet!")

processing fold # 0
Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 model_3 (Functional)           (None, None, 768)    101677056   ['input_1[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 lambda (Lambda)                (None, 768)          0           ['model

KeyboardInterrupt: ignored

In [ ]:
model.save('bert200-k10_6310.h5')

In [ ]:
import pickle
with open('bert200_k10_results.pickle', 'wb') as f:
    pickle.dump(results, f)

In [ ]:
# convert the history.history dict to a pandas DataFrame:  
import pandas as pd   
hist_df = pd.DataFrame(history.history) 

# save to json:  
hist_json_file = 'history200_k10.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

# or save to csv: 
hist_csv_file = 'history200_k10.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

## 載入模型

In [8]:
model = build_model()  
model.load_weights('bert200-k10_6310.h5')

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 model_3 (Functional)           (None, None, 768)    101677056   ['input_1[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 lambda (Lambda)                (None, 768)          0           ['model_3[0][0]']          

## 實際上來用用看，看看它對於輸出的預測是什麼：

---



In [10]:
sentiment_text = '大智慧的人，他的心非常的安詳，他過著世界太平的日子。應該無所著，生起清淨心。為什麼？世間，不值得我們執著。不值得我們這樣痛苦，煎熬自己。這是很愚痴的人，但問題是習氣難改。'

ids, segments = tokenizer.encode(sentiment_text, max_len=512)

my_X_test = [np.array([ids]), np.array([segments])]

predict = model.predict(my_X_test)
LabelRank = np.argsort(predict)
print(predict)
print(LabelRank)

[[9.6177295e-02 2.8489102e-02 1.0365417e-03 7.9962951e-01 2.5823817e-03
  5.6395256e-05 2.5041523e-05 2.1959697e-03 6.8125874e-03 1.4742593e-03
  2.8604770e-04 5.4539654e-02 3.7794972e-03 9.1733108e-04 9.8394428e-04
  6.0082297e-04 1.1074905e-04 3.0287652e-04]]
[[ 6  5 16 10 17 15 13 14  2  9  7  4 12  8  1 11  0  3]]


In [ ]:
Buddha_Dic = {'0':'一切有為法，如夢幻泡影','1':'凡所有相，皆是虛妄','2':'若真修道人，不見世間過','3':'應無所住，而生其心',
              '4':'過去心不可得，現在心不可得','5':'色不異空，空不異色','6':'不生不滅，不垢不淨','7':'不取於相，如如不動',
              '8':'如來者，無所從來','9':'無有定法，如來可說','10':'無有恐怖，遠離顛倒夢想','11':'法尚應捨，何況非法',
              '12':'本來無一物，何處惹塵埃','13':'一念悟時，眾生是佛','14':'何期自性，本自清淨','15':'不是幡動，仁者心動',
              '16':'真語者，實語者','17':'是法平等，無有高下'}  